In [1]:
import praw
import os
import requests
import csv
import pandas as pd
import urllib
import re
import random

In [2]:
CLIENT_ID = "tfGXJma3moRz7NGvrNENtA"
CLIENT_SECRET = "mzMSqFgs4zBMk8ILetit8LNj84g5_A"
USER_AGENT = "Scrape473 by u/Rainet473"
reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    user_agent=USER_AGENT,
)

In [3]:
print(reddit.user.me())

None


### Function to Extract Posts
Reddit posts are extracted including media when a keyword and a subreddit is mentioned.

In [4]:
def extract_info(subreddit_name, search_keyword, image_no, received_posts, num_posts=10):
    # Create a directory to save the downloaded media
    save_directory = 'media_downloads'
    os.makedirs(save_directory, exist_ok=True)
    print("Subreddit_name:", subreddit_name)
    print("Search_keyword:", search_keyword)
    
    # Iterate through the subreddit posts
    subreddit = reddit.subreddit(subreddit_name)
    try:
        search_results = list(subreddit.search(search_keyword, limit=num_posts))
        print(len(search_results))
    except:
        return image_no
    
    non_media_posts = 3
    for post in search_results:   ##Considering new posts for the time being
        
        if post.url in received_posts:
            print("Duplicate Post found")
            continue
        else:
            received_posts.add(post.url)
            
        video_url = None
        image_url = None
        media_url = None
        media = []
        obtained_data = [search_keyword, str(post.url), str(post.title.encode('ascii', 'ignore'))]
        flag = False
        ##To check for media other than images
        if post.media:
            try:
                if 'reddit_video' in post.media:
                    # Download videos
                    video_url = post.media['reddit_video']['fallback_url']
                    file_extension = '.mp4'
                    save_path = os.path.join(save_directory, f'{post.id}{file_extension}')
                    response = requests.get(video_url)
                    with open(save_path, 'wb') as f:
                        f.write(response.content)
                    flag = True
                    print(f"Video downloaded: {post.id}{file_extension}")
                    media.append(f'{post.id}{file_extension}')
                    #obtained_data.append(f'{post.id}{file_extension}')
                elif 'oembed' in post.media:
                    # Download media from oembed
                    media_url = post.media['oembed']['url']
                    file_extension = os.path.splitext(media_url)[1]
                    save_path = os.path.join(save_directory, f'{post.id}{file_extension}')
                    response = requests.get(media_url)
                    with open(save_path, 'wb') as f:
                        f.write(response.content)
                    flag = True
                    print(f"Media downloaded: {post.id}{file_extension}")
                    media.append(f'{post.id}{file_extension}')
                    #obtained_data.append(f'{post.id}{file_extension}')
            except:
                continue
        
        ##If post has a single image
        if post.url.endswith(('.jpg', '.jpeg', '.png', '.gif')):
            # Download images
            image_url = post.url
            file_extension = os.path.splitext(image_url)[1]
            save_path = os.path.join(save_directory, f'Image{image_no}{file_extension}')
            response = requests.get(image_url)
            with open(save_path, 'wb') as f:
                f.write(response.content)
            Flag = True
            print(f"Image downloaded: Image{image_no}")
            media.append(f'Image{image_no}{file_extension}')
            #obtained_data.append(f'{post.id}{file_extension}')
            image_no+=1
        try:
        ##If post has multiple images
            if post.media_metadata:
                # Iterate over the media metadata
                for media_id, media_data in post.media_metadata.items():
                    # Check if the media type is an image
                    if media_data.get('e') == 'Image':
                        # Get the image URL
                        image_url = media_data['s']['u']

                        # Download the image
                        response = requests.get(image_url)
                        if response.status_code == 200:
                            # Save the image to a file
                            save_path = os.path.join(save_directory, f'Image{image_no}.jpg')
                            with open(f'{save_path}', 'wb') as file:
                                file.write(response.content)
                            flag = True
                            print(f"Image{image_no} downloaded.")  
                            media.append(f'Image{image_no}.jpg')
                            image_no+=1
        except:
            pass
        ##To obtain images from comments
        try:
            for comment in post.comments.list():
                # Check if the comment has a body and contains URLs
                if comment.body and comment.body_html and comment.author == post.author.name:
                    # Extract URLs from the comment body
                    urls = re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', comment.body)
                    #print(urls)
                    # Download the images from the URLs
                    for url in urls:
                        # Check if the URL points to an image
                        if '.jpg' in url or '.jpeg' in url or '.png' in url or '.gif' in url:
                            try:
                                # Send a GET request to download the image
                                response = requests.get(url)
                                if response.status_code == 200:
                                    # Save the image to a file
                                    filename = url.split('/')[-1]
                                    filename = filename.split('?')[0]
                                    filename = filename.split(')')[0]
                                    file_extension = '.jpg' if '.jpg' in url else '.jpeg' if '.jpeg' in url else '.png' if '.png' in url else '.gif' if '.gif' in url else ''
                                    save_path = os.path.join(save_directory, f'Image{image_no}{file_extension}')
                                    with open(save_path, 'wb') as file:
                                        file.write(response.content)
                                    flag = True
                                    print(f"Image downloaded: Image{image_no}")
                                    media.append(f'Image{image_no}{file_extension}')
                                    image_no+=1
                            except requests.exceptions.RequestException as e:
                                print(f"Failed to download image from {url}: {e}")
        except:
            pass
        obtained_data.append(", ".join(media))
        
        if flag==False:
            if non_media_posts>0:
                non_media_posts-=1
            else:
                continue
            
        if len(str(post.selftext))>0:
            obtained_data.append(str(post.selftext.encode('ascii', 'ignore')))
        try:    
        #print(obtained_data)
            with open("reddit.csv", "a") as csvfile:
                csvwriter = csv.writer(csvfile)
                csvwriter.writerow(obtained_data)
            print("--------------------")
        except:
            if flag==False:
                non_media_posts+=1
        
    return image_no

In [13]:
Items = ["penicillin", "anticonvulsant", "allopurinol",
         "corticosteroid", "antimicrobial", "NSAID", "Tetracycline", "barbiturate", "codeine", "carbamezepine",
         "acetaminophen", "Advil"]
#"Drug reaction", "Unintended drug effects", 
Relating = ["rash" , "swelling", "itch", "problem", "side effect", "eyes", "vomit", "constipation"]

Subreddits = ["ADHD", "AskDocs", "DermatologyQuestions", "medicine", "DiagnoseMe", "Allergy", "Allergies", "bipolar",
              "gout", "psoriasis", "sciatica", "askdentists", "acne", "AcneScars", "Accutane", "mentalillnes",
              "mentalhealth", "skin", "headaches"]

In [5]:
##In case you want to rewrite
def rewrite(csv_name):
    with open(csv_name, "w") as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Keyword', 'Link', 'Title', 'Media', 'Body'])

In [ ]:
image_no = 0
for key in Items:
    for relating_key in Relating:
        for subreddit in Subreddits:
            image_no = extract_info(subreddit, key+" "+relating_key, image_no, num_posts = 10)

Subreddit_name: ADHD
Search_keyword: penicillin rash
1
--------------------
Subreddit_name: AskDocs
Search_keyword: penicillin rash
Subreddit_name: DermatologyQuestions
Search_keyword: penicillin rash
4
Image0 downloaded.
Image1 downloaded.
Image2 downloaded.
Image3 downloaded.
--------------------
Image4 downloaded.
Image5 downloaded.
Image6 downloaded.
Image7 downloaded.
Image8 downloaded.
Image9 downloaded.
--------------------
Image10 downloaded.
Image11 downloaded.
Image12 downloaded.
Image13 downloaded.
Image14 downloaded.
Image15 downloaded.
Image16 downloaded.
Image17 downloaded.
--------------------
Image18 downloaded.
--------------------
Subreddit_name: medicine
Search_keyword: penicillin rash
Subreddit_name: DiagnoseMe
Search_keyword: penicillin rash
6
Image downloaded: Image19
--------------------
--------------------
--------------------
Image20 downloaded.
Image21 downloaded.
Image22 downloaded.
Image23 downloaded.
Image24 downloaded.
Image25 downloaded.
Image26 download

Subreddit_name: askdentists
Search_keyword: penicillin side effect
10
--------------------
Image downloaded: Image54
--------------------
--------------------
Subreddit_name: acne
Search_keyword: penicillin side effect
10
Image55 downloaded.
Image56 downloaded.
Image57 downloaded.
Image58 downloaded.
Image59 downloaded.
--------------------
Image60 downloaded.
Image61 downloaded.
Image62 downloaded.
Image63 downloaded.
Image64 downloaded.
--------------------
--------------------
--------------------
--------------------
Subreddit_name: AcneScars
Search_keyword: penicillin side effect
10
--------------------
--------------------
--------------------
Image downloaded: Image65
Subreddit_name: Accutane
Search_keyword: penicillin side effect
10
--------------------
--------------------
--------------------
Image66 downloaded.
Image67 downloaded.
Image68 downloaded.
Image69 downloaded.
Image70 downloaded.
Image71 downloaded.
--------------------
Image72 downloaded.
Image73 downloaded.
Image

0
Subreddit_name: AskDocs
Search_keyword: anticonvulsant itch
Subreddit_name: DermatologyQuestions
Search_keyword: anticonvulsant itch
0
Subreddit_name: medicine
Search_keyword: anticonvulsant itch
Subreddit_name: DiagnoseMe
Search_keyword: anticonvulsant itch
0
Subreddit_name: Allergy
Search_keyword: anticonvulsant itch
0
Subreddit_name: Allergies
Search_keyword: anticonvulsant itch
0
Subreddit_name: bipolar
Search_keyword: anticonvulsant itch
1
--------------------
Subreddit_name: gout
Search_keyword: anticonvulsant itch
Subreddit_name: psoriasis
Search_keyword: anticonvulsant itch
0
Subreddit_name: sciatica
Search_keyword: anticonvulsant itch
0
Subreddit_name: askdentists
Search_keyword: anticonvulsant itch
0
Subreddit_name: acne
Search_keyword: anticonvulsant itch
0
Subreddit_name: AcneScars
Search_keyword: anticonvulsant itch
0
Subreddit_name: Accutane
Search_keyword: anticonvulsant itch
0
Subreddit_name: mentalillnes
Search_keyword: anticonvulsant itch
Subreddit_name: mentalhealt

0
Subreddit_name: skin
Search_keyword: anticonvulsant vomit
Subreddit_name: headaches
Search_keyword: anticonvulsant vomit
0
Subreddit_name: ADHD
Search_keyword: anticonvulsant constipation
0
Subreddit_name: AskDocs
Search_keyword: anticonvulsant constipation
Subreddit_name: DermatologyQuestions
Search_keyword: anticonvulsant constipation
0
Subreddit_name: medicine
Search_keyword: anticonvulsant constipation
Subreddit_name: DiagnoseMe
Search_keyword: anticonvulsant constipation
0
Subreddit_name: Allergy
Search_keyword: anticonvulsant constipation
0
Subreddit_name: Allergies
Search_keyword: anticonvulsant constipation
0
Subreddit_name: bipolar
Search_keyword: anticonvulsant constipation
0
Subreddit_name: gout
Search_keyword: anticonvulsant constipation
Subreddit_name: psoriasis
Search_keyword: anticonvulsant constipation
0
Subreddit_name: sciatica
Search_keyword: anticonvulsant constipation
0
Subreddit_name: askdentists
Search_keyword: anticonvulsant constipation
0
Subreddit_name: acne


--------------------
--------------------
--------------------
--------------------
Subreddit_name: AcneScars
Search_keyword: allopurinol side effect
10
--------------------
--------------------
--------------------
Image downloaded: Image141
Subreddit_name: Accutane
Search_keyword: allopurinol side effect
10
--------------------
--------------------
--------------------
Image142 downloaded.
Image143 downloaded.
Image144 downloaded.
Image145 downloaded.
Image146 downloaded.
Image147 downloaded.
--------------------
Image148 downloaded.
Image149 downloaded.
Image150 downloaded.
--------------------
Image downloaded: Image151
Image downloaded: Image152
Image153 downloaded.
Image154 downloaded.
--------------------
Subreddit_name: mentalillnes
Search_keyword: allopurinol side effect
Subreddit_name: mentalhealth
Search_keyword: allopurinol side effect
10
--------------------
--------------------
--------------------
Subreddit_name: skin
Search_keyword: allopurinol side effect
Subreddit_nam

...

Subreddit_name: DiagnoseMe
Search_keyword: barbiturate eyes
0
Subreddit_name: Allergy
Search_keyword: barbiturate eyes
0
Subreddit_name: Allergies
Search_keyword: barbiturate eyes
0
Subreddit_name: bipolar
Search_keyword: barbiturate eyes
0
Subreddit_name: gout
Search_keyword: barbiturate eyes
Subreddit_name: psoriasis
Search_keyword: barbiturate eyes
Subreddit_name: sciatica
Search_keyword: barbiturate eyes
0
Subreddit_name: askdentists
Search_keyword: barbiturate eyes
0
Subreddit_name: acne
Search_keyword: barbiturate eyes
0
Subreddit_name: AcneScars
Search_keyword: barbiturate eyes
0
Subreddit_name: Accutane
Search_keyword: barbiturate eyes
0
Subreddit_name: mentalillnes
Search_keyword: barbiturate eyes
Subreddit_name: mentalhealth
Search_keyword: barbiturate eyes
0
Subreddit_name: skin
Search_keyword: barbiturate eyes
Subreddit_name: headaches
Search_keyword: barbiturate eyes
0
Subreddit_name: ADHD
Search_keyword: barbiturate vomit
0
Subreddit_name: AskDocs
Search_keyword: barbitur

Subreddit_name: mentalhealth
Search_keyword: codeine problem
10
--------------------
--------------------
--------------------
Subreddit_name: skin
Search_keyword: codeine problem
Subreddit_name: headaches
Search_keyword: codeine problem
0
Subreddit_name: ADHD
Search_keyword: codeine side effect
10
--------------------
--------------------
--------------------
Subreddit_name: AskDocs
Search_keyword: codeine side effect
Subreddit_name: DermatologyQuestions
Search_keyword: codeine side effect
10
--------------------
--------------------
--------------------
Subreddit_name: medicine
Search_keyword: codeine side effect
Subreddit_name: DiagnoseMe
Search_keyword: codeine side effect
10
--------------------
--------------------
Image431 downloaded.
Image432 downloaded.
--------------------
--------------------
Image downloaded: Image433
Subreddit_name: Allergy
Search_keyword: codeine side effect
10
--------------------
Image downloaded: Image434
--------------------
--------------------
Subre

Subreddit_name: DermatologyQuestions
Search_keyword: carbamezepine swelling
0
Subreddit_name: medicine
Search_keyword: carbamezepine swelling
Subreddit_name: DiagnoseMe
Search_keyword: carbamezepine swelling
0
Subreddit_name: Allergy
Search_keyword: carbamezepine swelling
0
Subreddit_name: Allergies
Search_keyword: carbamezepine swelling
0
Subreddit_name: bipolar
Search_keyword: carbamezepine swelling
0
Subreddit_name: gout
Search_keyword: carbamezepine swelling
Subreddit_name: psoriasis
Search_keyword: carbamezepine swelling
Subreddit_name: sciatica
Search_keyword: carbamezepine swelling
0
Subreddit_name: askdentists
Search_keyword: carbamezepine swelling
0
Subreddit_name: acne
Search_keyword: carbamezepine swelling
0
Subreddit_name: AcneScars
Search_keyword: carbamezepine swelling
0
Subreddit_name: Accutane
Search_keyword: carbamezepine swelling
0
Subreddit_name: mentalillnes
Search_keyword: carbamezepine swelling
Subreddit_name: mentalhealth
Search_keyword: carbamezepine swelling
0


Subreddit_name: psoriasis
Search_keyword: carbamezepine vomit
Subreddit_name: sciatica
Search_keyword: carbamezepine vomit
0
Subreddit_name: askdentists
Search_keyword: carbamezepine vomit
0
Subreddit_name: acne
Search_keyword: carbamezepine vomit
0
Subreddit_name: AcneScars
Search_keyword: carbamezepine vomit
0
Subreddit_name: Accutane
Search_keyword: carbamezepine vomit
0
Subreddit_name: mentalillnes
Search_keyword: carbamezepine vomit
Subreddit_name: mentalhealth
Search_keyword: carbamezepine vomit
0
Subreddit_name: skin
Search_keyword: carbamezepine vomit
Subreddit_name: headaches
Search_keyword: carbamezepine vomit
0
Subreddit_name: ADHD
Search_keyword: carbamezepine constipation
0
Subreddit_name: AskDocs
Search_keyword: carbamezepine constipation
Subreddit_name: DermatologyQuestions
Search_keyword: carbamezepine constipation
0
Subreddit_name: medicine
Search_keyword: carbamezepine constipation
Subreddit_name: DiagnoseMe
Search_keyword: carbamezepine constipation
0
Subreddit_name:

--------------------
Subreddit_name: medicine
Search_keyword: acetaminophen side effect
Subreddit_name: DiagnoseMe
Search_keyword: acetaminophen side effect
10
--------------------
--------------------
Image498 downloaded.
Image499 downloaded.
--------------------
--------------------
Image downloaded: Image500
Subreddit_name: Allergy
Search_keyword: acetaminophen side effect
10
--------------------
Image downloaded: Image501
--------------------
--------------------
Subreddit_name: Allergies
Search_keyword: acetaminophen side effect
10
--------------------
--------------------
--------------------
Subreddit_name: bipolar
Search_keyword: acetaminophen side effect
10
--------------------
--------------------
--------------------
Image downloaded: Image502
Image downloaded: Image503
Subreddit_name: gout
Search_keyword: acetaminophen side effect
Subreddit_name: psoriasis
Search_keyword: acetaminophen side effect
Subreddit_name: sciatica
Search_keyword: acetaminophen side effect
10
-------

### Removing Duplicates from extracted posts

After extracting the posts from reddit. It was observed that some posts were repeated multiple times as well as their images were also downloaded multiple times. This is due to the presence of common keywords(provided in list) in the reddit posts.

In [9]:
posts = pd.read_csv('reddit.csv')
Links = {}
rows = []

for i in range(len(posts)):
    link = posts['Link'][i]
    keyword = posts['Keyword'][i]
    title = posts['Title'][i]
    media = "" if pd.isna(posts['Media'][i]) else posts['Media'][i]
    medias = [] if pd.isna(posts['Media'][i]) else posts['Media'][i].split(', ')
    body = posts['Body'][i]
    
    if link in Links:
        Links[link] += medias
    else:
        Links[link] = []
        rows.append([keyword, link, title, media, body])

print("Original Posts:",len(rows))
saved_directory = 'media_downloads'

total_duplicate_media = 0

for link in Links:
    if len(Links[link])>0:
        for image_name in Links[link]:
            if 'Image524' in image_name:
                print(image_name)
            path = os.path.join(saved_directory, image_name)
            if os.path.exists(path):
                os.remove(path)
            else:
                if os.path.exists(path+".jpg"):
                    os.remove(path+".jpg")
                elif os.path.exists(path+".jpeg"):
                    os.remove(path+".jpeg")
                elif os.path.exists(path+".png"):
                    os.remove(path+".png")
                elif os.path.exists(path+".gif"):
                    os.remove(path+".gif")
                else:
                    pass
            #if '.jpg' not in path or '.jpeg' not in path or '.png' not in path or '.gif' not in path:
            #os.remove(path)
    total_duplicate_media += len(Links[link])

random.shuffle(rows)    
rewrite('reddit.csv')
with open("reddit.csv", "a") as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerows(rows)
    
    
print("Total Duplicate Posts Removed: ", len(posts)-len(rows))    
print("Total Duplicate Images Removed: ",total_duplicate_media)

Original Posts: 3359
Total Duplicate Posts Removed:  0
Total Duplicate Images Removed:  0


In [6]:
def collect_links(csvname):
    dataframe = pd.read_csv(csvname)
    Links = set()
    for link in dataframe['Link']:
        if link not in Links:
            Links.add(link)
        else:
            pass
    return Links

In [7]:
Items = ["lipitor", "lisinopril",
         "lorazepam", "losartan", "lyrica", "macrobid", "meloxicam", "methadone", "methocarbamol", "methylprednisolone",
         "metronidazole", "morphine", "nabumetone", "namenda", "neurontin", "nortriptyline", "oxycontin", "phentermine",
         "plaquenil", "pradaxa", "prednisone", "prilosec", "promethazine", "prozac", "tramadol", "trazodone", "trulicity",
         "victoza", "wellbutrin"]
 
Relating = ["rash" , "blood", "vomit", "eyes", "nausea", "pain", "discharge", ""]

Subreddits = ["ADHD", "AskDocs", "DermatologyQuestions", "medicine", "DiagnoseMe", "Allergies", "bipolar",  "Dentistry", "acne",
              "Accutane", "mentalhealth", "medical_advice", "insomnia", "ambien", "antidepressants", "Anxiety",
              "depression", "aspergers", "miagraine", "benzodiazepines"]

In [8]:
image_no = 1373
posts = collect_links('reddit.csv')
for key in Items:
    if key=='lipitor':
        for subreddit in Subreddits[16:]:
            image_no = extract_info(subreddit, key, image_no, posts, num_posts = 20)
    else:
        for subreddit in Subreddits:
            image_no = extract_info(subreddit, key, image_no, posts, num_posts = 20)
            
print(image_no)
    

Subreddit_name: depression
Search_keyword: lipitor
2
--------------------
--------------------
Subreddit_name: aspergers
Search_keyword: lipitor
0
Subreddit_name: miagraine
Search_keyword: lipitor
Subreddit_name: benzodiazepines
Search_keyword: lipitor
0
Subreddit_name: ADHD
Search_keyword: lisinopril
20
--------------------
--------------------
--------------------
Subreddit_name: AskDocs
Search_keyword: lisinopril
20
--------------------
--------------------
--------------------
Subreddit_name: DermatologyQuestions
Search_keyword: lisinopril
3
--------------------
Image1373 downloaded.
Image1374 downloaded.
--------------------
--------------------
Subreddit_name: medicine
Search_keyword: lisinopril
14
--------------------
--------------------
--------------------
Subreddit_name: DiagnoseMe
Search_keyword: lisinopril
20
--------------------
--------------------
--------------------
Duplicate Post found
Subreddit_name: Allergies
Search_keyword: lisinopril
1
--------------------
Subred

1
Duplicate Post found
Subreddit_name: bipolar
Search_keyword: lyrica
20
--------------------
--------------------
--------------------
Image downloaded: Image1395
Duplicate Post found
Image downloaded: Image1396
Subreddit_name: Dentistry
Search_keyword: lyrica
6
--------------------
Image downloaded: Image1397
--------------------
--------------------
Subreddit_name: acne
Search_keyword: lyrica
0
Subreddit_name: Accutane
Search_keyword: lyrica
1
Image1398 downloaded.
Image1399 downloaded.
Image1400 downloaded.
Image1401 downloaded.
Image1402 downloaded.
Image1403 downloaded.
--------------------
Subreddit_name: mentalhealth
Search_keyword: lyrica
20
--------------------
--------------------
--------------------
Duplicate Post found
Subreddit_name: medical_advice
Search_keyword: lyrica
20
Duplicate Post found
--------------------
--------------------
--------------------
Duplicate Post found
Duplicate Post found
Duplicate Post found
Duplicate Post found
Subreddit_name: insomnia
Search_

Video downloaded: if03ve.mp4
--------------------
Subreddit_name: ADHD
Search_keyword: methocarbamol
0
Subreddit_name: AskDocs
Search_keyword: methocarbamol
20
--------------------
--------------------
--------------------
Duplicate Post found
Duplicate Post found
Subreddit_name: DermatologyQuestions
Search_keyword: methocarbamol
0
Subreddit_name: medicine
Search_keyword: methocarbamol
0
Subreddit_name: DiagnoseMe
Search_keyword: methocarbamol
3
--------------------
--------------------
--------------------
Subreddit_name: Allergies
Search_keyword: methocarbamol
0
Subreddit_name: bipolar
Search_keyword: methocarbamol
3
--------------------
--------------------
--------------------
Subreddit_name: Dentistry
Search_keyword: methocarbamol
2
--------------------
--------------------
Subreddit_name: acne
Search_keyword: methocarbamol
0
Subreddit_name: Accutane
Search_keyword: methocarbamol
0
Subreddit_name: mentalhealth
Search_keyword: methocarbamol
1
--------------------
Subreddit_name: me

--------------------
Subreddit_name: medicine
Search_keyword: morphine
20
--------------------
--------------------
--------------------
Duplicate Post found
Image downloaded: Image1491
--------------------
Media downloaded: 8e0zug
--------------------
Subreddit_name: DiagnoseMe
Search_keyword: morphine
20
--------------------
Image downloaded: Image1492
--------------------
--------------------
Duplicate Post found
Duplicate Post found
Duplicate Post found
Subreddit_name: Allergies
Search_keyword: morphine
2
--------------------
--------------------
Subreddit_name: bipolar
Search_keyword: morphine
20
--------------------
Image downloaded: Image1493
--------------------
--------------------
Subreddit_name: Dentistry
Search_keyword: morphine
20
--------------------
--------------------
--------------------
Subreddit_name: acne
Search_keyword: morphine
1
--------------------
Subreddit_name: Accutane
Search_keyword: morphine
2
--------------------
--------------------
Subreddit_name: ment

Duplicate Post found
Subreddit_name: Allergies
Search_keyword: nortriptyline
0
Subreddit_name: bipolar
Search_keyword: nortriptyline
13
--------------------
--------------------
--------------------
Subreddit_name: Dentistry
Search_keyword: nortriptyline
4
--------------------
Duplicate Post found
--------------------
--------------------
Subreddit_name: acne
Search_keyword: nortriptyline
0
Subreddit_name: Accutane
Search_keyword: nortriptyline
0
Subreddit_name: mentalhealth
Search_keyword: nortriptyline
17
--------------------
--------------------
--------------------
Duplicate Post found
Duplicate Post found
Subreddit_name: medical_advice
Search_keyword: nortriptyline
20
Duplicate Post found
--------------------
Duplicate Post found
--------------------
--------------------
Subreddit_name: insomnia
Search_keyword: nortriptyline
19
--------------------
--------------------
--------------------
Duplicate Post found
Duplicate Post found
Subreddit_name: ambien
Search_keyword: nortriptyli

8
--------------------
--------------------
--------------------
Subreddit_name: DiagnoseMe
Search_keyword: pradaxa
0
Subreddit_name: Allergies
Search_keyword: pradaxa
0
Subreddit_name: bipolar
Search_keyword: pradaxa
0
Subreddit_name: Dentistry
Search_keyword: pradaxa
1
Duplicate Post found
Subreddit_name: acne
Search_keyword: pradaxa
0
Subreddit_name: Accutane
Search_keyword: pradaxa
0
Subreddit_name: mentalhealth
Search_keyword: pradaxa
0
Subreddit_name: medical_advice
Search_keyword: pradaxa
3
--------------------
--------------------
--------------------
Subreddit_name: insomnia
Search_keyword: pradaxa
0
Subreddit_name: ambien
Search_keyword: pradaxa
0
Subreddit_name: antidepressants
Search_keyword: pradaxa
0
Subreddit_name: Anxiety
Search_keyword: pradaxa
0
Subreddit_name: depression
Search_keyword: pradaxa
0
Subreddit_name: aspergers
Search_keyword: pradaxa
0
Subreddit_name: miagraine
Search_keyword: pradaxa
Subreddit_name: benzodiazepines
Search_keyword: pradaxa
0
Subreddit_nam

14
--------------------
--------------------
--------------------
Subreddit_name: ADHD
Search_keyword: promethazine
15
Duplicate Post found
--------------------
--------------------
--------------------
Subreddit_name: AskDocs
Search_keyword: promethazine
20
Duplicate Post found
--------------------
--------------------
--------------------
Subreddit_name: DermatologyQuestions
Search_keyword: promethazine
2
Duplicate Post found
Video downloaded: y4nq2g.mp4
--------------------
Subreddit_name: medicine
Search_keyword: promethazine
6
Duplicate Post found
--------------------
--------------------
--------------------
Subreddit_name: DiagnoseMe
Search_keyword: promethazine
11
Duplicate Post found
--------------------
Duplicate Post found
--------------------
Duplicate Post found
--------------------
Subreddit_name: Allergies
Search_keyword: promethazine
11
Duplicate Post found
--------------------
--------------------
--------------------
Subreddit_name: bipolar
Search_keyword: promethazin

--------------------
--------------------
Duplicate Post found
--------------------
Duplicate Post found
Image downloaded: Image1691
Subreddit_name: Dentistry
Search_keyword: tramadol
20
Duplicate Post found
Duplicate Post found
--------------------
--------------------
--------------------
Duplicate Post found
Duplicate Post found
Subreddit_name: acne
Search_keyword: tramadol
1
Duplicate Post found
Subreddit_name: Accutane
Search_keyword: tramadol
3
Duplicate Post found
--------------------
--------------------
Subreddit_name: mentalhealth
Search_keyword: tramadol
20
Duplicate Post found
--------------------
--------------------
--------------------
Subreddit_name: medical_advice
Search_keyword: tramadol
20
Duplicate Post found
Duplicate Post found
--------------------
--------------------
--------------------
Duplicate Post found
Subreddit_name: insomnia
Search_keyword: tramadol
20
Duplicate Post found
--------------------
--------------------
--------------------
Duplicate Post foun

20
Duplicate Post found
Image1704 downloaded.
Image1705 downloaded.
--------------------
Duplicate Post found
--------------------
Duplicate Post found
--------------------
--------------------
Duplicate Post found
Duplicate Post found
Duplicate Post found
Duplicate Post found
Duplicate Post found
Subreddit_name: Allergies
Search_keyword: wellbutrin
8
Duplicate Post found
--------------------
--------------------
--------------------
Subreddit_name: bipolar
Search_keyword: wellbutrin
20
Duplicate Post found
Image1706 downloaded.
Image1707 downloaded.
--------------------
--------------------
--------------------
--------------------
Image downloaded: Image1708
Duplicate Post found
Subreddit_name: Dentistry
Search_keyword: wellbutrin
2
--------------------
Image downloaded: Image1709
--------------------
Subreddit_name: acne
Search_keyword: wellbutrin
5
Duplicate Post found
--------------------
--------------------
--------------------
Subreddit_name: Accutane
Search_keyword: wellbutrin

{1}


{1, 6, 7, 9, 10}


{1: 8}
